In [ ]:
#multi instance simulation, taking into account different weather conditions, thrust curves, oxidizer total mass uncertainty. The output values for far-out report parameters.

In [ ]:
from datetime import datetime
from rocketpy import EnvironmentAnalysis, Environment, Rocket, Environment, Flight, LiquidMotor, Fluid, CylindricalTank, MassFlowRateBasedTank, MassBasedTank
from math import exp
from datetime import datetime, timedelta
from CoolProp.CoolProp import PropsSI
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import excel_sheet_functions as ex
import setup
import config as cfg
from rocketpy.plots.compare import CompareFlights

In [ ]:
env_gefs = Environment(
    date=(2025, 5, 30, 12),
    latitude=35.35,
    longitude=-117.81,
    elevation=621,
)

env_gefs.set_atmospheric_model(type="Ensemble", file="GEFS")

In [ ]:
env_era5 = Environment(
    date=(2024, 5, 30, 12),
    latitude=35.35,
    longitude=-117.81,
    elevation=621,
    max_expected_height=6000,
)
filename = "data_stream-oper_stepType-instant.nc"
env_era5.set_atmospheric_model(
    type="Reanalysis",
    file=filename,
    dictionary="ECMWF",
)
# env_era5.plots.atmospheric_model()
# env_era5.prints.all()

In [ ]:
def get_env(date, latitude, longitude, elevation, max_expected_height):
    env = Environment(
        date=date,
        latitude=latitude,
        longitude=longitude,
        elevation=elevation,
        max_expected_height=max_expected_height,
    )
    env.set_atmospheric_model(
        type="Reanalysis",
        file="data_stream-oper_stepType-instant.nc",
        dictionary="ECMWF",
    )
    return env

In [ ]:
# Far-out constants
LATITUDE = 35.35
LONGITUDE = -117.81
ELEVATION = 621
MAX_EXPECTED_HEIGHT = 6000

In [ ]:
ox_liq, ox_gas, fuel_liq, fuel_gas = setup.create_fluids()

In [ ]:
dates = [
    (2016, 5, 28, 8),
    (2016, 5 ,28, 12),
    (2016, 5, 28, 16),
    (2016, 5, 29, 8),
    (2016, 5, 29, 12),
    (2016, 5, 29, 16),
    (2016, 5, 30, 8),
    (2016, 5, 30, 12),
    (2016, 5, 30, 16),
    (2017, 5, 28, 8),
    (2017, 5, 28, 12),
    (2017, 5, 28, 16),
    (2017, 5, 29, 8),
    (2017, 5, 29, 12),
    (2017, 5, 29, 16),
    (2017, 5, 30, 8),
    (2017, 5, 30, 12),
    (2017, 5, 30, 16),
    (2018, 5, 28, 8),
    (2018, 5 ,28, 12),
    (2018, 5, 28, 16),
    (2018, 5, 29, 8),
    (2018, 5, 29, 12),
    (2018, 5, 29, 16),
    (2018, 5, 30, 8),
    (2018, 5, 30, 12),
    (2018, 5, 30, 16),
    (2019, 5, 28, 8),
    (2019, 5 ,28, 12),
    (2019, 5, 28, 16),
    (2019, 5, 29, 8),
    (2019, 5, 29, 12),
    (2019, 5, 29, 16),
    (2019, 5, 30, 8),
    (2019, 5, 30, 12),
    (2019, 5, 30, 16),
    (2020, 5, 28, 8),
    (2020, 5 ,28, 12),
    (2020, 5, 28, 16),
    (2020, 5, 29, 8),
    (2020, 5, 29, 12),
    (2020, 5, 29, 16),
    (2020, 5, 30, 8),
    (2020, 5, 30, 12),
    (2020, 5 ,30 ,16),
    (2021, 5, 28, 8),
    (2021, 5 ,28, 12),
    (2021, 5, 28, 16),
    (2021, 5, 29, 8),
    (2021, 5, 29, 12),
    (2021, 5, 29, 16),
    (2021, 5, 30, 8),
    (2021, 5, 30, 12),
    (2021, 5 ,30 ,16),
    (2022, 5, 28, 8),
    (2022, 5 ,28, 12),
    (2022, 5, 28, 16),
    (2022, 5, 29, 8),
    (2022, 5, 29, 12),
    (2022, 5, 29, 16),
    (2022, 5, 30, 8),
    (2022, 5 ,30 ,12),
    (2022 ,5 ,30 ,16),
    (2023, 5, 28, 8),
    (2023, 5 ,28, 12),
    (2023, 5, 28, 16),
    (2023, 5, 29, 8),
    (2023, 5, 29, 12),
    (2023, 5, 29, 16),
    (2023, 5, 30, 8),
    (2023 ,5 ,30 ,12),
    (2023 ,5 ,30 ,16),
    (2024, 5, 28, 8),
    (2024, 5 ,28, 12),
    (2024, 5, 28, 16),
    (2024, 5, 29, 8),
    (2024, 5 ,29 ,12),
    (2024 ,5 ,29 ,16),
    (2024 ,5 ,30 ,8),
    (2024 ,5 ,30 ,12),
    (2024 ,5 ,30 ,16),
    (2025, 5, 28, 8),
    (2025, 5 ,28, 12),
    (2025, 5, 28, 16),
    (2025, 5, 29, 8),
    (2025, 5 ,29 ,12),
    (2025 ,5 ,29 ,16),
    (2025 ,5 ,30 ,8),
    (2025 ,5 ,30 ,12),
]

In [ ]:
thrust_curve_multiplier = [
    0.9,
    0.92,
    0.95,
    0.97,
    1.0,
    1.02,
    1.05,
    1.07,
    1.1,
]

In [ ]:
total_oxidizer_mass_and_piston_position = [
    (18, 0.85),
    (17, 0.80),
    (16, 0.75),
    (15, 0.70),
]

In [ ]:
#thrust curves
th_curve_df = pd.read_csv("data/thrust_curve_basic.csv", header=None, names=["time", "thrust"], index_col=False)
for multiplier in thrust_curve_multiplier:
    th_curve_df[f"thrust_{multiplier}"] = th_curve_df["thrust"] * multiplier
    th_curve_df.loc[:, ["time", f"thrust_{multiplier}"]].to_csv(f"data/thrust_curve_{int(multiplier*100)}.csv", index=False, header=False)

In [ ]:
# NOTE: These empty list initializations are only needed for the old sequential loop below.
# The optimized parallel version creates these lists directly from results.

# dane do excela
rail_departure_velocities = []
twrs = []
max_static_margins = []
min_static_margins = []
max_accelerations = []
max_speeds = []
max_mach_numbers = []
max_dynamic_pressures = []
distances_from_pad = []
max_yaw_moments = []
max_pitch_moments = []
landing_coordinates = []

In [ ]:

date = dates[0]
total_oxidizer_mass, piston_position = total_oxidizer_mass_and_piston_position[0]

env = get_env(date, LATITUDE, LONGITUDE, ELEVATION, MAX_EXPECTED_HEIGHT)
for thrust_multiplier in thrust_curve_multiplier:
    ox_tank, fuel_tank = setup.create_tanks(total_ox_mass=total_oxidizer_mass, piston_pos=piston_position)
    liq_motor = setup.create_motor(
    tanks=(ox_tank, fuel_tank),
    thrust_curve_file=f"data/thrust_curve_{int(thrust_multiplier*100)}.csv",
    burn_time=(1, cfg.BURN_TIME)
    )
    rocket = setup.create_rocket(motor=liq_motor)
    flight = setup.create_flight(rocket=rocket, env=env, inclination=87, heading=0, rail_length=15.24)
    rail_departure_velocities.append(ex.rail_departure_velocity_in_ft_per_sec(flight))
    twrs.append(ex.average_thrust_during_rail_phase(flight=flight, motor=liq_motor, rocket=rocket))
    max_static_margins.append(ex.max_static_margin(flight))
    min_static_margins.append(ex.min_static_margin(flight))
    max_accelerations.append(ex.max_acceleration_power_on_in_g(flight))
    max_speeds.append(ex.max_speed(flight))
    max_mach_numbers.append(ex.max_mach(flight))
    max_dynamic_pressures.append(ex.max_dynamic_pressure(flight))
    distances_from_pad.append(ex.distance_from_pad(flight))
    max_yaw_moments.append(ex.max_yaw_moment(flight))
    max_pitch_moments.append(ex.max_pitch_moment(flight))
    landing_coordinates.append((flight.x_impact, flight.y_impact))
            





In [ ]:
date = dates[0]
thrust_curve_multiplier_var = thrust_curve_multiplier[0]
total_oxidizer_mass, piston_position = total_oxidizer_mass_and_piston_position[0]
env = get_env(date, LATITUDE, LONGITUDE, ELEVATION, MAX_EXPECTED_HEIGHT)
ox_tank, fuel_tank = setup.create_tanks(total_ox_mass=total_oxidizer_mass, piston_pos=piston_position)
liq_motor = setup.create_motor(
    tanks=(ox_tank, fuel_tank),
    thrust_curve_file=f"data/thrust_curve_{int(thrust_curve_multiplier_var*100)}.csv",
    burn_time=(1, cfg.BURN_TIME),
)
rocket = setup.create_rocket(motor=liq_motor)
flight1 = setup.create_flight(rocket=rocket, env=env, inclination=87, heading=0, rail_length=15.24)

thrust_curve_multiplier_var2 = thrust_curve_multiplier[-1]
liq_motor = setup.create_motor(
    tanks=(ox_tank, fuel_tank),
    thrust_curve_file=f"data/thrust_curve_{int(thrust_curve_multiplier_var2*100)}.csv",
    burn_time=(1, cfg.BURN_TIME),
)
rocket = setup.create_rocket(motor=liq_motor)
flight2 = setup.create_flight(rocket=rocket, env=env, inclination=87, heading=0, rail_length=15.24)
flight1.plots.trajectory_3d()
flight2.plots.trajectory_3d()


In [ ]:
comparison = CompareFlights([flight1, flight2])
comparison.all()

In [ ]:
# creating flight objects for max, min thrust curvem max, min total oxidizer mass
min_thrust_multiplier = thrust_curve_multiplier[0]
max_thrust_multiplier = thrust_curve_multiplier[-1]
min_total_oxidizer_mass, min_piston_position = total_oxidizer_mass_and_piston_position[-1]
max_total_oxidizer_mass, max_piston_position = total_oxidizer_mass_and_piston_position[0]
env = get_env(date, LATITUDE, LONGITUDE, ELEVATION, MAX_EXPECTED_HEIGHT)
min_ox_tank, min_fuel_tank = setup.create_tanks(total_ox_mass=min_total_oxidizer_mass, piston_pos=min_piston_position)
max_ox_tank, max_fuel_tank = setup.create_tanks(total_ox_mass=max_total_oxidizer_mass, piston_pos=max_piston_position)
min_liq_motor_with_min_tanks = setup.create_motor(
    tanks=(min_ox_tank, min_fuel_tank),
    thrust_curve_file=f"data/thrust_curve_{int(min_thrust_multiplier*100)}.csv",
    burn_time=(1, cfg.BURN_TIME),
)
max_liq_motor_with_max_tanks = setup.create_motor(
    tanks=(max_ox_tank, max_fuel_tank),
    thrust_curve_file=f"data/thrust_curve_{int(max_thrust_multiplier*100)}.csv",
    burn_time=(1, cfg.BURN_TIME),
)
min_liq_motor_with_max_tanks = setup.create_motor(
    tanks=(max_ox_tank, max_fuel_tank),
    thrust_curve_file=f"data/thrust_curve_{int(min_thrust_multiplier*100)}.csv",
    burn_time=(1, cfg.BURN_TIME),
)
max_liq_motor_with_min_tanks = setup.create_motor(
    tanks=(min_ox_tank, min_fuel_tank),
    thrust_curve_file=f"data/thrust_curve_{int(max_thrust_multiplier*100)}.csv",
    burn_time=(1, cfg.BURN_TIME),
)
r1 = setup.create_rocket(motor=min_liq_motor_with_min_tanks)
r2 = setup.create_rocket(motor=max_liq_motor_with_max_tanks)
r3 = setup.create_rocket(motor=min_liq_motor_with_max_tanks)
r4 = setup.create_rocket(motor=max_liq_motor_with_min_tanks)

f1 = setup.create_flight(rocket=r1, env=env, inclination=87, heading=0, rail_length=15.24)
f2 = setup.create_flight(rocket=r2, env=env, inclination=87, heading=0, rail_length=15.24)
f3 = setup.create_flight(rocket=r3, env=env, inclination=87, heading=0, rail_length=15.24)
f4 = setup.create_flight(rocket=r4, env=env, inclination=87, heading=0, rail_length=15.24)
comparison = CompareFlights([f1, f2, f3, f4])

In [ ]:
comparison.trajectories_3d(legend=True)